# Prevent Colab from auto disconnecting
## copy and paste the folowing code to console
function ClickConnect(){
    console.log("Clicked on connect button"); 
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)

## stop function
var startClickConnect = function startClickConnect(){
    var clickConnect = function clickConnect(){
        console.log("Connnect Clicked - Start");
        document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
        console.log("Connnect Clicked - End"); 
    };

    var intervalId = setInterval(clickConnect, 60000);

    var stopClickConnectHandler = function stopClickConnect() {
        console.log("Connnect Clicked Stopped - Start");
        clearInterval(intervalId);
        console.log("Connnect Clicked Stopped - End");
    };

    return stopClickConnectHandler;
};

var stopClickConnect = startClickConnect();

# Import

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from PIL import Image
import json
import matplotlib.pyplot as plt
import os
import pandas as pd
import time
from IPython.display import display
import csv

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.get_device_name(device))

cpu


RuntimeError: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data resize and augmentation

In [5]:
root_dir = '/content/drive/My Drive/VRDL/HW1'
train_dir = root_dir + '/training_data/'
test_dir = root_dir + '/testing_data/'
info = pd.read_csv(root_dir + '/training_labels.csv')
info_id = info.iloc[:,0]
info_label = info.iloc[:,1]
classes = {}
i = 0
for label in info_label:
  if label not in classes:
    classes[label] = i
    i += 1

im_size = 400
train_trans = transforms.Compose([transforms.Resize((im_size, im_size)),
                  transforms.RandomHorizontalFlip(),
                  transforms.RandomRotation(15),
                  transforms.ToTensor(),
                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
test_trans = transforms.Compose([transforms.Resize((im_size, im_size)),
                 transforms.ToTensor(),
                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

class CarTrain(Dataset):
  def __init__(self, id, label, transform=None):
    self.id = id
    self.labels = label
    self.transform = transform
  
  def __len__(self):
    return len(self.id)
  
  def __getitem__(self, idx):
    img = Image.open(train_dir + str(self.id[idx]).zfill(6) + '.jpg').convert('RGB')
    label = classes[self.labels[idx]]

    if self.transform:
      img = self.transform(img)

    return img, label

class CarTest(Dataset):
  def __init__(self, root_dir, transform=None):
    self.root_dir = root_dir
    self.info = os.listdir(root_dir)
    self.transform = transform
  
  def __len__(self):
    return len(self.info)

  def __getitem__(self, idx):
    img = Image.open(self.root_dir + self.info[idx]).convert('RGB')
    id = self.info[idx][0:6]

    if self.transform:
      img = self.transform(img)
      
    return img, id

b_size = 128

train_data = CarTrain(info_id, info_label, transform=train_trans)
test_data = CarTest(root_dir=test_dir, transform=test_trans)
trainloader = DataLoader(train_data, batch_size=b_size, shuffle=True)
testloader = DataLoader(test_data, batch_size=32, shuffle=False)

# Model create

In [22]:
model_cars = models.resnet34(pretrained=True)
num_ftrs = model_cars.fc.in_features
model_cars.fc = nn.Linear(num_ftrs, 196)

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model_cars.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(params=model_cars.parameters(), lr=0.0001)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)
num_epochs = 10

# Load Model

In [30]:
def load_checkpoint(filepath):
  checkpoint = torch.load(filepath)
    
  #model.load_state_dict(checkpoint['state_dict'])
  model_cars.load_state_dict(checkpoint['state_dict'], strict=False)
  #model_cars.class_to_idx = checkpoint['class_to_idx']
    
  return model_cars
myModel = load_checkpoint(root_dir + '/tai.pth')

# Testing Model

In [31]:
def predict(model):
  print('----------Start Predicting----------')
  begin = time.time()
  res = []
  model.eval()
  batch_no = 1
  with torch.no_grad():
    for img, id in testloader:
      since = time.time()
      img = img.to(device)

      outputs = model(img)
      _, predicted = torch.max(outputs.data, 1)
      
      for i in range(len(predicted)):
        pred = []
        pred.append(id[i])
        pred.append(list(classes)[predicted[i]])
        res.append(pred)
      
      if batch_no % 10 == 0:
        print('Batch {}~{} finished, time needed: {}'.format(batch_no-9, batch_no, time.time()-since))
      batch_no += 1

    print('Predicting Finished elapsed time: {}'.format(time.time()-begin))
  return res

results = predict(myModel)

----------Start Predicting----------
Batch 1~10 finished, time needed: 0.20189714431762695
Batch 11~20 finished, time needed: 0.20283031463623047
Batch 21~30 finished, time needed: 0.20432543754577637
Batch 31~40 finished, time needed: 0.20334196090698242
Batch 41~50 finished, time needed: 0.20340871810913086
Batch 51~60 finished, time needed: 0.20476794242858887
Batch 61~70 finished, time needed: 0.2043468952178955
Batch 71~80 finished, time needed: 0.20347023010253906
Batch 81~90 finished, time needed: 0.20492911338806152
Batch 91~100 finished, time needed: 0.20543575286865234
Batch 101~110 finished, time needed: 0.20432615280151367
Batch 111~120 finished, time needed: 0.205322265625
Batch 121~130 finished, time needed: 0.20650124549865723
Batch 131~140 finished, time needed: 0.20556378364562988
Batch 141~150 finished, time needed: 0.2045290470123291
Predicting Finished elapsed time: 113.47908234596252


In [32]:
import csv

with open(root_dir + '/result.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['id', 'label'])
  writer.writerows(results)
print('result saved')

result saved


# Prediction completed!!!

In [ ]:
assert(False)

# Training Parameters Setting

In [5]:
def train_model(model, criterion, optimizer, scheduler, n_epochs = 5):
  losses = []
  accuracies = []
  #test_accuracies = []
  model.to('cuda')
  model.train()
  begin = time.time()
  print('----------Training Begin----------')
  for epoch in range(n_epochs):
    since = time.time()
    running_loss = 0.0
    running_correct = 0.0
    for inputs, labels in trainloader:

      # get the inputs and assign them to cuda
      #inputs, labels = data
      inputs = inputs.to('cuda')
      labels = labels.to('cuda')
      optimizer.zero_grad()
            
      # forward + backward + optimize
      outputs = model.forward(inputs)
      _, predicted = torch.max(outputs.data, 1)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
            
      # calculate the loss/acc later
      running_loss += loss.item()
      running_correct += (labels==predicted).sum().item()

    epoch_duration = time.time()-since
    epoch_loss = running_loss/len(trainloader)
    epoch_acc = 100/b_size*running_correct/len(trainloader)
    print("Epoch %s/%s, duration: %d s, loss: %.4f, acc: %.4f" % (epoch+1, n_epochs, epoch_duration, epoch_loss, epoch_acc))
        
    losses.append(epoch_loss)
    accuracies.append(epoch_acc)

    checkpoint = {'state_dict': model.state_dict(),
        'model': model,
        'opt_state': optimizer.state_dict,}

    torch.save(checkpoint, '/content/drive/My Drive/VRDL/HW1/checkpoint.pth')
    
  print('Finished Training, total time: {}'.format(time.time()-begin))
  return model, losses, accuracies

# Training Model

In [25]:
myModel, training_losses, training_accs = train_model(model_cars, criterion, optimizer, lrscheduler, n_epochs=10)

----------Training Begin----------
Epoch 1/10, duration: 391 s, loss: 0.0651, acc: 98.1623
Epoch 2/10, duration: 384 s, loss: 0.0564, acc: 98.2688
Epoch 3/10, duration: 381 s, loss: 0.0504, acc: 98.3842
Epoch 4/10, duration: 379 s, loss: 0.0503, acc: 98.3132
Epoch 5/10, duration: 380 s, loss: 0.0567, acc: 98.2333
Epoch 6/10, duration: 391 s, loss: 0.0340, acc: 98.6861
Epoch 7/10, duration: 394 s, loss: 0.0286, acc: 98.8015
Epoch 8/10, duration: 389 s, loss: 0.0295, acc: 98.8370
Epoch 9/10, duration: 392 s, loss: 0.0227, acc: 98.8370
Epoch 10/10, duration: 393 s, loss: 0.0279, acc: 98.7305
Finished Training, total time: 3891.673525571823


# Save Model

In [29]:
checkpoint = {'state_dict': myModel.state_dict(),
        'model': myModel,
        'opt_state': optimizer.state_dict,}

torch.save(checkpoint, '/content/drive/My Drive/VRDL/HW1/checkpoint.pth')